# Day 3 : Binary Diagnostic

## Part 1 : Puzzle Description

The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the **power consumption**.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the **gamma rate** and the **epsilon rate**). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the **most common bit in the corresponding position** of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```

Considering only the first bit of each number, there are five `0` bits and seven `1` bits. Since the most common bit is `1`, the first bit of the gamma rate is `1`.

The most common second bit of the numbers in the diagnostic report is `0`, so the second bit of the gamma rate is `0`.

The most common value of the third, fourth, and fifth bits are `1`, `1`, and `0`, respectively, and so the final three bits of the gamma rate are `110`.

So, the gamma rate is the binary number `10110`, or **`22`** in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is `01001`, or **`9`** in decimal. Multiplying the gamma rate (`22`) by the epsilon rate (`9`) produces the power consumption, **`198`**.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. **What is the power consumption of the submarine?** (Be sure to represent your answer in decimal, not binary.)

## Load Diagnostic Report

In [1]:
# with open("example", "r") as file:
with open("input", "r") as file:
    report = file.read().splitlines()

print("Report Entries:", len(report))

Report Entries: 1000


## Helper Functions

In [2]:
def get_bit_count(report: list, bit_len: int = 0, shift_right: int = 0) -> list:

    if bit_len < 1:
        bit_len = get_bitlen(report)

    if shift_right > 0:
        bit_len -= shift_right

    # Gamma counts per binary position start at zero.
    # Afer final tallies, positives counts equal 1, negatives equal 0.
    # Ignore tie count as challenge does not list criteria for how to handle it.
    gamma_counts = [0] * bit_len

    # Loop over report lines (1000 times).
    for reading in report:

        read_int = int(reading, 2)
        if shift_right > 0:
            read_int = read_int >> shift_right

        # Loop over each bit (bit_len times).
        for i in range(bit_len):
            # Check bit i starting from the right moving to the left.
            # Interation Round 1 EX: 010i where i = 0 or 1
            # Interation Round 2 EX: 01i0 where i = 0 or 1
            if read_int & (2**i) == 0:
                gamma_counts[-i - 1] -= 1
            else:
                gamma_counts[-i - 1] += 1

    return gamma_counts

def get_gamma(gamma_counts: list) -> int:

    # Convert to gamma reading (binary with most common bits).
    gamma = 0
    for bit_count in gamma_counts:
        # If currnet bit count is positive set right most gamma bit to 1.
        # Otherwise leave it as a zero.
        if bit_count > 0:
            gamma = gamma | 1

        # Now shift bits to the left.
        # New right most bit will be zero and updated next interation.
        gamma = gamma << 1

    # Off by one shift left. Shift once back right.
    gamma = gamma >> 1

    return gamma

def get_epsilon(gamma: int, bit_len: int) -> int:
    bit_len_mask = (2**bit_len) - 1
    return (gamma ^ bit_len_mask)  # XOR gamma for epsilon.

def get_bitlen(report: list) -> tuple:
    bit_len = len(report[0])
    return bit_len


## Part 1 : Puzzle Solution

In [3]:
bit_len = get_bitlen(report)
print("Bit Length:", bit_len)

counts = get_bit_count(report)
print("Gamma Counts:", counts)

# Gamma, Epsilon, and Power Consumption.
gamma = get_gamma(counts)
epsilon = get_epsilon(gamma, bit_len)
power = gamma * epsilon

print("Gamma Reading:", gamma, f"\t\t({'{0:b}'.format(gamma).zfill(bit_len)})")

print("Epsilon Reading:", epsilon, f"\t\t({'{0:b}'.format(epsilon).zfill(bit_len)})")

print("Power Consumption:", power, f"\t({'{0:b}'.format(power).zfill(bit_len * 2)})")


Bit Length: 12
Gamma Counts: [18, 44, -34, 16, 50, 22, -36, -24, -24, 28, 34, 50]
Gamma Reading: 3527 		(110111000111)
Epsilon Reading: 568 		(001000111000)
Power Consumption: 2003336 	(000111101001000110001000)


## Part 2 : Puzzle Description

Next, you should verify the **life support rating**, which can be determined by multiplying the **oxygen generator rating** by the **CO2 scrubber rating**.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and **consider just the first bit** of those numbers. Then:

- Keep only numbers selected by the **bit criteria** for the type of rating value for which you are searching. Discard numbers which do not match the bit criteria.
- If you only have one number left, stop; this is the rating value for which you are searching.
- Otherwise, repeat the process, considering the next bit to the right.

The **bit criteria** depends on which type of rating value you want to find:

- To find **oxygen generator rating**, determine the **most common** value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a **`1`** in the position being considered.
- To find **CO2 scrubber rating**, determine the **least common** value (`0` or `1`) in the current bit position, and keep only numbers with that bit in that position. If `0` and `1` are equally common, keep values with a **`0`** in the position being considered.

For example, to determine the **oxygen generator rating** value using the same example diagnostic report from above:

- Start with all 12 numbers and consider only the first bit of each number. There are more `1` bits (7) than `0` bits (5), so keep only the 7 numbers with a `1` in the first position: `11110`, `10110`, `10111`, `10101`, `11100`, `10000`, and `11001`.
- Then, consider the second bit of the 7 remaining numbers: there are more `0` bits (4) than `1` bits (3), so keep only the 4 numbers with a `0` in the second position: `10110`, `10111`, `10101`, and `10000`.
- In the third position, three of the four numbers have a `1`, so keep those three: `10110`, `10111`, and `10101`.
- In the fourth position, two of the three numbers have a `1`, so keep those two: `10110` and `10111`.
- In the fifth position, there are an equal number of `0` bits and `1` bits (one each). So, to find the **oxygen generator rating**, keep the number with a `1` in that position: `10111`.
- As there is only one number left, stop; the **oxygen generator rating** is `10111`, or **`23`** in decimal.

Then, to determine the **CO2 scrubber rating** value from the same example above:

- Start again with all 12 numbers and consider only the first bit of each number. There are fewer `0` bits (5) than `1` bits (7), so keep only the 5 numbers with a `0` in the first position: `00100`, `01111`, `00111`, `00010`, and `01010`.
- Then, consider the second bit of the 5 remaining numbers: there are fewer `1` bits (2) than `0` bits (3), so keep only the 2 numbers with a `1` in the second position: `01111` and `01010`.
- In the third position, there are an equal number of `0` bits and `1` bits (one each). So, to find the **CO2 scrubber rating**, keep the number with a `0` in that position: `01010`.
- As there is only one number left, stop; the **CO2 scrubber rating** is `01010`, or **`10`** in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (`23`) by the CO2 scrubber rating (`10`) to get **`230`**.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. **What is the life support rating of the submarine?** (Be sure to represent your answer in decimal, not binary.)

## Part 2 : Puzzle Solution

In [4]:
bit_len = get_bitlen(report)
print("Bit Length:", bit_len)

ratings = {"O2 Generator": 0, "CO2 Scrubber": 0}

for ls_type in ratings.keys():

    ls = report

    start = 0
    end = start + 1

    print("Analyzing:", ls_type)

    while len(ls) > 1:

        # Get bit counts one bit column at a time using a sliding window.
        counts = get_bit_count(
            ls, bit_len=(bit_len - start), shift_right=(bit_len - end)
        )

        # Get gamma if O2 and epsilon if CO2.
        if ls_type.startswith("O2"):
            if counts[0] == 0:
                common_bit = 1
            else:
                common_bit = get_gamma(counts)
        else:
            if counts[0] == 0:
                common_bit = 0
            else:
                gamma = get_gamma(counts)
                common_bit = get_epsilon(gamma, 1)  # Only ever one bit at a time.

        # print("Common Bit:", common_bit)

        # Filter all those that don't match the most/least common bit.
        ls = [reading for reading in ls if reading[start:end] == str(common_bit)]
        # print(ls)

        # Move sliding window for next loop iteration.
        start += 1
        end += 1

    # List type done, save for later.
    ratings[ls_type] = int(ls[0], 2)

    # print(f"\nRating: {ls_type} = {ratings[ls_type]}")

# Done, print results.
print(f"\n{('-' * 35)}")
print("O2 Generator Rating: ", ratings["O2 Generator"])
print("CO2 Scrubber Rating: ", ratings["CO2 Scrubber"])
print("Life Support Rating: ", (ratings["O2 Generator"] * ratings["CO2 Scrubber"]))


Bit Length: 12
Analyzing: O2 Generator
Analyzing: CO2 Scrubber

-----------------------------------
O2 Generator Rating:  2547
CO2 Scrubber Rating:  737
Life Support Rating:  1877139


## After Thoughts

Part 1 does not have criteria for what happens if there is a tie in `1`'s and `0`'s for a given bit? I think the input is designed to avoid this issue but it forces me to make a decision about how to handle this without clear criteria in the challenge. I've chosen to ignore it and seem to get the correct answer.

Part 1 was rewritten significantly from the previous commit here. Mostly to make Part 2 easier.

I did this in bitwise operators as much as possible mainly just because this whole challenge was about manipulating bits. In a real project I would not rely on bitwise operators as much. While it is possible to speed things up with bitwise operations, it makes the code look horrendous and pratically incomprhensible to most humans. Developers working with code like this are far more likely to make shift errors or off by one errors that are hard to find and review by other developers.